# Experimentation

Loading the datasets and cleaning. The following datasets are expected:
* `locations_clean_user_location.tsv`: The original provided list of raw locations with corresponding number of occurances
* In `/data`:
  * `cities1000.tsv`, cities with > 1000 pop. (GeoNames):
    * https://download.geonames.org/export/dump/cities1000.zip
    * Unzipped and renamed to `.tsv`
  * `countryInfo.tsv`, countries (GeoNames):
    * https://download.geonames.org/export/dump/countryInfo.txt
    * Unzipped and renamed to `.tsv`
  * `admin1CodesASCII.txt`, states and provinces (admin1) (GeoNames)
    * https://download.geonames.org/export/dump/admin1CodesASCII.txt

In [16]:
import os
import pandas as pd
import geopandas as gpd

current_dir = os.getcwd()
data_dir = "data"

## Load datasets

In [17]:
# Tweets user locations list
# Loading using pandas' read_csv (tab-deleted) to set 'tweet_id' dtype to int

tweets_user_locations = os.path.join(current_dir, "locations_clean_user_location.tsv")
df = pd.read_csv(tweets_user_locations, sep='\t', dtype={'tweet_id': int})
df.head(3)

,tweet_user_location,tweet_id
0,None,4994911
1,United States,190257
2,India,97652


In [18]:
# GeoNames (Cities with > 1000 inabitants)
# https://download.geonames.org/export/dump/cities1000.zip
# Loading using geopandas for geometry (usefulness tbd)
# NB: We can ignore 'DtypeWarning' as we don't need column 13

cities = os.path.join(current_dir, data_dir, "cities1000.tsv")
# cities_df = gpd.read_file(cities)
cities_df = pd.read_csv(cities, sep='\t',
            names=['geonameid', 'name', 'asciiname', 'altnames', 'latitude', 'longitude',
                   'featclass', 'featcode', 'country', 'cc2', 'admin1', 'admin2', 'admin3', 'admin4',
                   'population', 'elevation', 'gtopo30', 'timezone', 'moddate'],
            dtype={'admin3': str}) # can't set column 13
cities_df.head(3)

/Users/pndurette/.pyenv/versions/3.8.3/envs/chumblab/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,geonameid,name,asciiname,altnames,latitude,longitude,featclass,featcode,country,cc2,admin1,admin2,admin3,admin4,population,elevation,gtopo30,timezone,moddate
0,3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,02,NaN,NaN,NaN,1052,NaN,1721,Europe/Andorra,2012-11-03
1,3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,06,NaN,NaN,NaN,8022,NaN,921,Europe/Andorra,2013-11-23
2,3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,03,NaN,NaN,NaN,2363,2050.0,2106,Europe/Andorra,2008-06-09


In [19]:
# Alternate City names
# The cities_df dataframe has an 'altnames' column that are CSVs
# We transform it to a list then explode it to make searching easier
# (and reset the index). The new column 'altname' can be used to search.

cities_alt_df = cities_df.assign(
    altname=cities_df['altnames'].str.split(',')
    ).explode('altname').reset_index(drop=True)

cities_alt_df.head(3)

cities_alt_df[cities_alt_df['altname'] == 'NYC']

,geonameid,name,asciiname,altnames,latitude,longitude,featclass,featcode,country,cc2,admin1,admin2,admin3,admin4,population,elevation,gtopo30,timezone,moddate,altname
627577,5128581,New York City,New York City,"Aebura,Bandar Raya New York,Big Apple,Cathair ...",40.71427,-74.00597,P,PPL,US,NaN,NY,NaN,NaN,NaN,8175133,10.0,57,America/New_York,2019-09-23,NYC


In [20]:
# Test cities finding
# City
city_test = cities_df[(cities_df['name'] == 'London')]
city_test

# City & admin1
city_test = cities_df[(cities_df['name'] == 'London') & \
                      (cities_df['admin1'] == 'ENG')]
city_test

# City & admin1 & country
city_test = cities_df[(cities_df['name'] == 'London') & \
                      (cities_df['admin1'] == 'ENG') & \
                      (cities_df['country'] == 'GB')]
city_test

# City & country
city_test = cities_df[(cities_df['name'] == 'London') & \
                      (cities_df['country'] == 'GB')]
city_test

# City (with the largest population)
cities_df[(cities_df['name'] == 'London')].nlargest(1, ['population']) 
city_test

# City with alternative names

city_test = cities_df[(cities_df['name'] == 'London')].copy()
city_test

# city_test.apply(lambda x: x.astype('str').str.split(',')).explode('altnames')
# city_test.apply(lambda x: x.astype(str).str.split(',').explode()).reset_index(

# city_test = cities_df[(cities_df['name'] == 'LON') | ('LON' in cities_df['altnames'].str.split(',') )]
# city_test = cities_df[('LON' in cities_df['altnames'].str.split(',') )]
# city_test

# df = df[~df['tweet_user_location'].isin(tweet_user_location_discard)]

,geonameid,name,asciiname,altnames,latitude,longitude,featclass,featcode,country,cc2,admin1,admin2,admin3,admin4,population,elevation,gtopo30,timezone,moddate
13904,6058560,London,London,"Landona,London,Londonas,Londono,YXU,leondeon,l...",42.98339,-81.23304,P,PPL,CA,NaN,08,NaN,NaN,NaN,346765,NaN,252,America/Toronto,2012-08-19
48817,2643743,London,London,"ILondon,LON,Lakana,Landan,Landen,Ljondan,Llund...",51.50853,-0.12574,P,PPLC,GB,NaN,ENG,GLA,NaN,NaN,7556900,NaN,25,Europe/London,2019-09-18
118957,4119617,London,London,"Haddoxburg,London",35.32897,-93.25296,P,PPL,US,NaN,AR,115,90813,NaN,1046,116.0,121,America/Chicago,2017-05-23
120898,4298960,London,London,"LOZ,Landon,London,Riceton,lndn,lndn kntaky,lu...",37.12898,-84.08326,P,PPLA2,US,NaN,KY,125,NaN,NaN,8126,378.0,379,America/New_York,2017-03-09
122937,4517009,London,London,"Landon,Limerick,London,New London,lndn,lndn a...",39.88645,-83.44825,P,PPLA2,US,NaN,OH,097,44674,NaN,10060,321.0,321,America/New_York,2017-05-23
132245,5367815,London,London,"London,Londres,New London",36.47606,-119.44318,P,PPL,US,NaN,CA,107,NaN,NaN,1869,91.0,93,America/Los_Angeles,2011-05-14


In [21]:
# GeoNames (Countries info)
# https://download.geonames.org/export/dump/countryInfo.txt
# Loading using pandas' read_csv (tab-deleted), ignore lines 1-48

countries = os.path.join(current_dir, data_dir, "countryInfo.tsv")
countries_df = pd.read_csv(countries, sep='\t', header=49)
countries_df.head(3)

,#ISO,ISO3,ISO-Numeric,fips,Country,Capital,Area(in sq km),Population,Continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid,neighbours,EquivalentFipsCode
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,.ae,AED,Dirham,971,NaN,NaN,"ar-AE,fa,en,hi,ur",290557,"SA,OM",NaN
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,.af,AFN,Afghani,93,NaN,NaN,"fa-AF,ps,uz-AF,tk",1149361,"TM,CN,IR,TJ,PK,UZ",NaN


In [22]:
# GeoNames (states and provinces, admin1)
# https://download.geonames.org/export/dump/admin1CodesASCII.txt
# Loading using pandas' read_csv (tab-deleted),
# Column names from https://download.geonames.org/export/dump/readme.txt
# 'code' is '<country>.<admin1 for country>'

admin1 = os.path.join(current_dir, data_dir, "admin1CodesASCII.txt")
admin1_df = pd.read_csv(admin1, sep='\t', names=['code', 'name', 'name ascii', 'geonameid'])
admin1_df.head(3)

,code,name,name ascii,geonameid
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131


In [23]:
admin1_df[admin1_df['name'] == 'Ontario']

,code,name,name ascii,geonameid
473,CA.08,Ontario,Ontario,6093943


In [24]:
# GeoNames 'admin1' (admin1_df) for Canadian provinces uses a 2-digit code
# Use postal abbreviation which people use

# CA.01	Alberta	Alberta	5883102
# CA.02	British Columbia	British Columbia	5909050
# CA.03	Manitoba	Manitoba	6065171
# CA.04	New Brunswick	New Brunswick	6087430
# CA.13	Northwest Territories	Northwest Territories	6091069
# CA.07	Nova Scotia	Nova Scotia	6091530
# CA.14	Nunavut	Nunavut	6091732
# CA.08	Ontario	Ontario	6093943
# CA.09	Prince Edward Island	Prince Edward Island	6113358
# CA.10	Quebec	Quebec	6115047
# CA.11	Saskatchewan	Saskatchewan	6141242
# CA.12	Yukon	Yukon	6185811
# CA.05	Newfoundland and Labrador	Newfoundland and Labrador	6354959

province_abbr = {
    'CA.01': 'CA.AB', # Alberta
    'CA.02': 'CA.BC', # British Columbia
    'CA.03': 'CA.MB', # Manitoba
    'CA.04': 'CA.NB', # New Brunswick
    'CA.05': 'CA.NL', # Newfoundland and Labrador
    'CA.07': 'CA.NS', # Nova Scotia
    'CA.08': 'CA.ON', # Ontario
    'CA.09': 'CA.PE', # Prince Edward Island
    'CA.10': 'CA.QC', # Quebec
    'CA.11': 'CA.SK', # Saskatchewan
    'CA.12': 'CA.YK', # Yukon
    'CA.13': 'CA.NT', # Northwest Territories
    'CA.14': 'CA.NU'  # Nunavut
}

new_provinces = admin1_df[admin1_df['code'].str.contains('^CA.')].copy()
new_provinces['code'] = new_provinces['code'].map(province_abbr)
admin1_df = pd.concat([admin1_df, new_provinces], ignore_index=True)
admin1_df[admin1_df['code'].str.contains('^CA.')]

# Add to cities too
# Query the cities with country CA and num code, copy,
# replace num code by letter code, then concat back
new_cities = pd.DataFrame([], columns=countries_df.columns)
for num_code, letter_code in province_abbr.items():
    country, num = tuple(num_code.split('.'))
    country, letter = tuple(letter_code.split('.'))
 
    alt_cities = cities_alt_df[(cities_alt_df['country'] == country) & \
                               (cities_alt_df['admin1'] == num)].copy()
    alt_cities['admin1'] = letter
    new_cities = pd.concat([new_cities, alt_cities], ignore_index=True)

cities_alt_df = pd.concat([cities_alt_df, new_cities], ignore_index=True)
cities_alt_df[cities_alt_df['admin1'] == 'ON']

,geonameid,name,asciiname,altnames,latitude,longitude,featclass,featcode,country,cc2,...,Continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,neighbours,EquivalentFipsCode
678465,5882600,Agincourt North,Agincourt North,NaN,43.80418,-79.27528,P,PPLX,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678466,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678467,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678468,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678469,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678470,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678471,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678472,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678473,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678474,5882873,Ajax,Ajax,"Adzhaks,Ehjdzhaks,Ejdzaks,ajaks,ajaks antaryw...",43.85012,-79.03288,P,PPL,CA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Test when there's more than 1 admin1

# admin1_df[admin1_df['code'].str.contains('^US.')]
# admin1_df[admin1_df['name'] == 'La Paz']
country_code = 'HN'
element = 'La Paz'
test1 = admin1_df[(admin1_df['name'] == element)].copy()
test1.loc[:, 'geonameid'] = 99
test11 = test1.head(1)
test1

,code,name,name ascii,geonameid
351,BO.04,La Paz,La Paz,99
1190,HN.12,La Paz,La Paz,99
3288,SV.06,La Paz,La Paz,99


In [26]:
# With both country-code and admin1-code
country_code = 'CA'
admin1_code = 'ON'
admin1_df[(admin1_df['code'].str.contains(f'^{country_code}.{admin1_code}'))]

# With only an admin1-code
admin1_code = 'FL'
admin1_df[(admin1_df['code'].str.contains(f'.{admin1_code}$'))]

,code,name,name ascii,geonameid
3660,US.FL,Florida,Florida,4155751


In [27]:
# Alternative names (huge DB)

# alts = os.path.join(current_dir, data_dir, "alternateNamesV2.txt")
# alts_df = pd.read_csv(alts, sep='\t',
#             names=['alternateNameId', 'geonameid', 'isolanguage', 'alternate name',
#                    'isPreferredName', 'isShortName', 'isColloquial', 'isHistoric', 'from', 'to'])
# alts_df.head(3)

In [28]:
# alts_df[(alts_df['isPreferredName'] == 1) & (alts_df['alternate name'] == 'République Démocratique Du Congo')]

In [29]:
# Add alternative country names (e.g. USA, UK, etc.)
# (we can't easily get alternative country names)
alternative_country_names = {
    6252001: ['USA', 'US','United States of America','America'], # United States
    2510769: ['España'], # [Kingdom of] Spain
    2635167: ['UK'],     # United Kingdom
    1861060: ['日本'],    # Japan
    298795: ['Türkiye'], # Turkey
    3469034: ['Brasil'], # Brazil
    3175395: ['Italia'], # Italy
    1694008: ['Republic of the Philippines'], # Philipines
    2921044: ['Deutschland'] # Germany
}

new_countries = pd.DataFrame([], columns=countries_df.columns)
for geo, alt_names in alternative_country_names.items():
    for name in alt_names:
        alt_country = countries_df[countries_df['geonameid'] == geo].copy()
        alt_country['Country'] = name
        new_countries = pd.concat([new_countries, alt_country], ignore_index=True)

countries_df = pd.concat([countries_df, new_countries], ignore_index=True)
        
# TODO: City alternartives
# And others like abbreviations (e.g. CDMX)
# Add to list of city alternatives
# #3527646: 'CDMX',   # Mexico City 

## Constants

In [30]:
# Discard specific 'tweet_user_location' strings
LOCATION_DISCARD = ['', 'none', '\\n', 'global', 'earth',
                    'planet earth', 'worldwide', 'everywhere',
                    'internet', 'en todas partes',
                    'europe', 'africa',
                    'world']

## Helper functions

In [31]:
# Displays the percentage of tweets that have a 'geonameid'
# Skipping the ones we know aren't valid (discards)
def print_geonameid_completeness(df):
    all_tweets = df[~df['tweet_user_location_copy'].isin(LOCATION_DISCARD)]['tweet_id'].sum()
    geonameid_tweets = df[df.geonameid.notnull()]['tweet_id'].sum()
    print(f'{geonameid_tweets/all_tweets*100:.3f}%')

# Show dataframe df where 'geonameid' is NaN and location
# is not known to be invalid (discards)
def show_nan(df):
    nan_df = df[(~df['tweet_user_location_copy'].isin(LOCATION_DISCARD)) & df['geonameid'].isnull()]
    print(f'Number of NaNs: {len(nan_df.index)}')
    return nan_df

 ## Clean

In [50]:
# Make a copy of 'tweet_user_location' so we leave the original intact
df['tweet_user_location_copy'] = df['tweet_user_location']

# Discard specific 'tweet_user_location' strings
# tweet_user_location_discard = ['None', '\\N']
# df = df[~df['tweet_user_location'].isin(tweet_user_location_discard)]

# Discard locations that don't exist more than 2 times
df = df[df['tweet_id'] > 2]

# Filter out emojis and other symbols
# * https://stackoverflow.com/a/49986645
# * https://www.ling.upenn.edu/courses/Spring_2003/ling538/UnicodeRanges.html (Unicode symbol ranges)
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emojis: emoticons
        u"\U0001F300-\U0001F5FF"  # emojis: symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # emojis: transport & map symbols
        u"\U00002700-\U000027BF"  # 'Dingbats' http://www.unicode.org/charts/PDF/U2700.pdf
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: deEmojify(x))

# Truncate leading and trailing spaces
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: x.strip())

# Filter out emoji flags, except if there's only one
# n.b. 1 emoji = 2 characters (i.e. '{2}')
def deFlag(text):
    regrex_oneflag = re.compile(pattern = u"^[\U0001F1E0-\U0001F1FF]{2}$", flags = re.UNICODE)
    regrex_allflags = re.compile(pattern = u"[\U0001F1E0-\U0001F1FF]+", flags = re.UNICODE)
    if regrex_oneflag.search(text) is not None:
        return text
    else:
        return regrex_allflags.sub(r'',text)
        
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: deFlag(x))
    
# Truncate trailing "," and "." characters
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: x.rstrip(','))
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].map(lambda x: x.rstrip('.'))

df

# pd.set_option('display.max_rows', 20000)
# df[df['tweet_user_location_copy'].str.match(r'[\U0001F1E0-\U0001F1FF]') == True]

,tweet_user_location,tweet_id,tweet_user_location_copy
0,None,4994911,None
1,United States,190257,United States
2,India,97652,India
3,"London, England",77542,"London, England"
4,USA,67336,USA
...,...,...,...
338210,N 52°27' 0'' / W 1°49' 0'',3,N 52°27' 0'' / W 1°49' 0''
338211,Villerupt-Luxembourg-Oslo-Stoc,3,Villerupt-Luxembourg-Oslo-Stoc
338212,Chicago ✈,3,Chicago
338213,Catch Me If You Can,3,Catch Me If You Can


## Clean: Lower case

In [18]:
# Make everything used for comparison lowercase for simplicity

# Locations
df['tweet_user_location_copy'] = df['tweet_user_location_copy'].str.lower()

# Cities
cities_alt_df['name'] = cities_alt_df['name'].str.lower()
cities_alt_df['asciiname'] = cities_alt_df['asciiname'].str.lower()
cities_alt_df['altname'] = cities_alt_df['altname'].str.lower()
cities_alt_df['admin1'] = cities_alt_df['admin1'].str.lower()

# Admin1
admin1_df['code'] = admin1_df['code'].str.lower()
admin1_df['name'] = admin1_df['name'].str.lower()
admin1_df['name ascii'] = admin1_df['name ascii'].str.lower()

# Countries
countries_df['Country'] = countries_df['Country'].str.lower()

## Countries

In [19]:
# Some locations are verbatim the name of a country, e.g.:
df[df['tweet_user_location_copy'] == 'canada']

,tweet_user_location,tweet_id,tweet_user_location_copy
18,Canada,40858,canada
1497,Canada,723,canada
1520,CANADA,712,canada
1594,canada,674,canada
2846,Canada 🇨🇦,352,canada
4756,Canada.,199,canada
5333,Canada,177,canada
9208,🇨🇦 Canada,100,canada
14493,Canada 🇨🇦,63,canada
17363,Canada🇨🇦,52,canada


In [20]:
# # df[df['tweet_user_location'].isin(simple_countries_df['Country'])]
# # simple_countries_df = countries_df[['#ISO','Country', 'geonameid']]#.set_index('Country')

# # Merge in country info (with goenameid) when there's an exact country match

# # Keep the columns of countries_df we need.
# simple_countries_df = countries_df[['#ISO','Country', 'geonameid']]
# #df = pd.merge(df, simple_countries_df, how='left', left_on='tweet_user_location_copy', right_on='Country')
# df

In [21]:
#print_geonameid_completeness(df)

## Cities

In [22]:
#simple_cities_df = cities_df[['geonameid', 'name', 'asciiname', 'altnames']]
# df = pd.merge(df, simple_cities_df, how='left', left_on='tweet_user_location', right_on='name')
#df_copy = df[df['geonameid'].isnull()]
#pd.merge(df_copy, simple_cities_df, how='left', left_on='tweet_user_location_copy', right_on='name')

# NB: this can't work b/c cities name (unlike countries) aren't unique, e.g. there's a lot of "London"

In [23]:
# df[df['tweet_user_location_copy'].str.count(',') > 2]

In [24]:
# test = "Toronto, Ontario, Canada, World"
# test = "Toronto, Canada"
# test.split(',')

In [25]:
# import numpy as np

# def split_fixed_parts(num_parts, location):
#     parts = location.split(',')
#     if num_parts > len(parts):
#         for i in range(num_parts - len(parts)):
#             parts.insert(0, None)
#     else:
#         for i in range(len(parts) - num_parts):
#             parts.pop(0)
#     return parts

# def parts_dict(num_parts, location):
#     parts = split_fixed_parts(num_parts, location)
#     return {f'el-{k}':parts[k] for k in range(num_parts)}

def split_parts(location):
    return location.split(',')

# print(split_fixed_parts(3, 'Toronto'))
print(split_parts('Toronto'))

['Toronto']


In [44]:
# Mini test dataset
# test_df = df[df['tweet_user_location_copy'].str.count(',') == 2].head(50)
test_df = df.head(20000).copy()
test_df

,tweet_user_location,tweet_id,tweet_user_location_copy
0,None,4994911,none
1,United States,190257,united states
2,India,97652,india
3,"London, England",77542,"london, england"
4,USA,67336,usa
...,...,...,...
19995,035,45,035
19996,Eisha,45,eisha
19997,兵庫県神戸市,45,兵庫県神戸市
19998,Cloud Nine,45,cloud nine


In [45]:
num_parts = 3

# https://stackoverflow.com/a/16242202
#test_df.tweet_user_location_copy.apply(lambda s: pd.Series(parts_dict(num_parts, s)))
#test_df = pd.concat([test_df, test_df.tweet_user_location_copy.apply(lambda s: pd.Series(parts_dict(num_parts, s)))], axis=1)

test_df['elements'] = test_df['tweet_user_location_copy'].map(lambda location: location.split(','))
test_df

,tweet_user_location,tweet_id,tweet_user_location_copy,elements
0,None,4994911,none,[none]
1,United States,190257,united states,[united states]
2,India,97652,india,[india]
3,"London, England",77542,"london, england","[london, england]"
4,USA,67336,usa,[usa]
...,...,...,...,...
19995,035,45,035,[035]
19996,Eisha,45,eisha,[eisha]
19997,兵庫県神戸市,45,兵庫県神戸市,[兵庫県神戸市]
19998,Cloud Nine,45,cloud nine,[cloud nine]


In [53]:
def get_country(countries_df, element):
    # Filter 'Country' field with 'element'
    country = countries_df[countries_df['Country'] == element]
    
    # No results
    if len(country) == 0:
        return None
    
    # There can be only one result
    return country


def get_admin1(admin1_df, element, country_code=None):
    if country_code is None:
        # admin1 matching name (or ascii name), or admin1 code w/o country
        admin1 = admin1_df[(admin1_df['name'] == element) | \
                           (admin1_df['name ascii'] == element) | \
                           (admin1_df['code'].str.contains(rf'.{element}$'))]
        
        # We duplicated some rows (e.g. to add Canadian provices by letter)
        # Drop geonameid duplicates. It doesn't matter which ones we keep
        admin1 = admin1.drop_duplicates(subset=['geonameid'])
        
        if len(admin1) == 0:
            # No results
            return None
    
        if len(admin1) > 1:
            # If it's an admin1 code
            if len(element) == 2:
                # If there's more than 1 admin1 codes and it's a 2 letter code,
                # take the one that is either USA or Canada
                admin1_ = admin1.copy()
                admin1 = admin1_[(admin1_['code'].str.contains(rf'us.{element}$')) | \
                                 (admin1_['code'].str.contains(rf'ca.{element}$'))]
                
                if len(admin1) == 1:
                    return admin1
                
                else:
                    # #ERROR:99
                    # This error happens when for an admin1 code,
                    # w/o a country, there is more than 1 admin1 row that is not USA or Canada
                    admin1 = admin1_df[admin1_df['code'].str.contains(rf'.{element}$')].copy()
                    admin1.loc[:, 'geonameid'] = 99
                    return admin1.head(1)
            
            # #ERROR:98
            # This error happens when, w/o a country, there is
            # more than 1 admin1 by that name/ascii name.
            # There is not enough data to infer which one.
            # e.g. "La Paz" district (Bolivia, Honduras, El Savador)
            admin1 = admin1_df[(admin1_df['name'] == element)].copy()
            admin1.loc[:, 'geonameid'] = 98
            return admin1.head(1)
    
    else:
        country_code = country_code.lower()
        # admin1 code matching <country_code>.<abbreviation>
        # or (admin1 code starts with <country_code> and (match name (or ascii name)))
        admin1 = admin1_df[
            (admin1_df['code'] == f'{country_code}.{element}') | \
                ((admin1_df['code'].str.contains(rf'^{country_code}.')) & \
                 ((admin1_df['name'] == element) | (admin1_df['name ascii'] == element)))]
        
        if len(admin1) == 0:
            # No results
            return None
        
        if len(admin1) > 1:
            # #ERROR:97
            # This error happens when, with a country, there is
            # more than 1 admin1 by that name/ascii name.
            admin1 = admin1_df[(admin1_df['name'] == element)].copy()
            admin1.loc[:, 'geonameid'] = 97
            return admin1.head(1)
    
    return admin1


def get_city(cities_df, element, admin1_code=None, country_code=None):
    # print(f'admin1_code: {admin1_code}, country_code: {country_code}')
    if admin1_code is None and country_code is None:
        cities = cities_df[(cities_df['altname'] == element)]
    
    elif admin1_code is None:
        cities = cities_df[(cities_df['altname'] == element) & \
                         (cities_df['country'] == country_code)]
    
    elif country_code is None:
        cities = cities_df[(cities_df['altname'] == element) & \
                           (cities_df['admin1'] == admin1_code)]
    
    else:
        cities = cities_df[(cities_df['altname'] == element) & \
                           (cities_df['admin1'] == admin1_code) & \
                           (cities_df['country'] == country_code)]
    
    if len(cities) == 0:
        # No results
        return None
    
    else:
        # More than one result,
        # take the city with the largest population.
        return cities.nlargest(1, ['population']) 

"""
Cases:

country

city, state/prov, country
city, state/prov
city, country
city

neighboorhood, city, country
neighboorhood, city

state/prov, country
state/prov
"""

# Filter out Regex 'metacharacters' (compile regex for performance)
# https://docs.python.org/3/howto/regex.html#matching-characters
meta_chars = ".^$*+?{}[]\|()"
meta_chars = [re.escape(i) for i in list(meta_chars)]
regrex_pattern = re.compile("|".join(meta_chars))

def infer_geonameid(elements):
    try:
        return compute_geonameid(elements)
    except ValueError:
        return np.nan

def compute_geonameid(elements):
    # Datasets
    # * countries_df
    # * admin1_df
    # * cities_df

    # Remove leading/trailing spaces, lowercase
    elements = [e.strip().lower() for e in elements]
    
    # Don't try to infer if element should be ignored
    if elements[0] in LOCATION_DISCARD:
        return np.nan
    
    # Filter out Regex 'metacharacters'
    elements = [regrex_pattern.sub(r'', e) for e in elements]
    
    # One item
    # TODO: Invert? Check city first, then state, then country?
    # e.g. New York is always the city, not the state.
    if len(elements) == 1:
        country = get_country(countries_df, elements[0])
        
        # "<country>" as-is
        if country is not None:
            return str(country['geonameid'].item())
    
        admin1 = get_admin1(admin1_df, elements[0])
    
        # "<state/province>" as-is
        if admin1 is not None:
            return str(admin1['geonameid'].item())

        city = get_city(cities_alt_df, elements[0])
        
        # "<city>" as-is
        if city is not None:
            return str(city['geonameid'].item())

    
    # Two items (0, 1)
    elif len(elements) == 2:
        
        country = get_country(countries_df, elements[1])
        
        # if element[1] is country:
        if country is not None:
            country_code = str(country['#ISO'].item())
            
            # Get admin1 (restrict to <country>)
            admin1 = get_admin1(admin1_df, elements[0], country_code=country_code)
    
            # if element[0] is <state/province> within <country>:
            if admin1 is not None:
                return str(admin1['geonameid'].item())
            
            # Get city (restrict to <country>)
            city = get_city(cities_alt_df, elements[0], country_code=country_code)
        
            # if element[0] is <city> within <country>:
            if city is not None:
                return str(city['geonameid'].item())
                
            # return country
            return str(country['geonameid'].item())
        
        
        admin1 = get_admin1(admin1_df, elements[1])
        
        # if element[1] is <state/province>:
        if admin1 is not None:
            
            # Format is '<COUNTRY_CODE>.<ADMIN1_CODE>'
            # Split it, then make it into a tuple
            country_code, admin1_code = tuple(str(admin1['code'].item()).split('.'))
            
            # Get city (restrict to <state/province>)
            city = get_city(cities_alt_df, elements[0], admin1_code=admin1_code)
            
            # if element[0] is <city> within <country>:
            if city is not None:
                return str(city['geonameid'].item())
                
            # return <state/province>
            return str(admin1['geonameid'].item())
         
        city = get_city(cities_alt_df, elements[1])
        
        # if element[1] is <city>:
        if city is not None:
            return str(city['geonameid'].item())
    
    # Three items
    elif len(elements) == 3:
        
        country = get_country(countries_df, elements[2])
        
        # if element[2] is country:
        if country is not None:
            country_code = str(country['#ISO'].item())

            # Get admin1 (restrict to <country>)
            admin1 = get_admin1(admin1_df, elements[1], country_code=country_code)
            
            # if element[1] is <state/province> within <country>:
            if admin1 is not None:
                
                # Format is '<COUNTRY_CODE>.<ADMIN1_CODE>'
                # Split it, then make it into a tuple
                country_code, admin1_code = tuple(str(admin1['code'].item()).split('.'))
                
                # Get city (restrict to <state/province>)
                city = get_city(cities_alt_df, elements[0], admin1_code=admin1_code)
                
                # if element[0] if <city> within <state/province>
                if city is not None:
                    # return <city>
                    return str(city['geonameid'].item())
                   
                # return <state/province>
                return str(admin1['geonameid'].item())
                
            # Get city (restrict to <state/province>)
            city = get_city(cities_alt_df, elements[1], country_code=country_code)
                
            # if element[1] is <city> within <country>:
            if city is not None:
                # return <city>
                return str(city['geonameid'].item())
                
            # return country
            return str(country['geonameid'].item())
            
        admin1 = get_admin1(admin1_df, elements[2])
        
        # if element[2] is <state/province>:
        if admin1 is not None:
            
            # Format is '<COUNTRY_CODE>.<ADMIN1_CODE>'
            # Split it, then make it into a tuple
            country_code, admin1_code = tuple(str(admin1['code'].item()).split('.'))
            
            # Get city (restrict to <state/province>)
            city = get_city(cities_alt_df, elements[1], admin1_code=admin1_code)
            
            # if element[1] if <city> within <state/province>
            if city is not None:
                # return <city>
                return str(city['geonameid'].item())
            
    return np.nan

In [46]:
#infer_geonameid(['ca'])
get_admin1(admin1_df, 'ca')
# element = "CA"
# admin1_df[(admin1_df['name'] == element) | \
#                            (admin1_df['name ascii'] == element) | \
#                            (admin1_df['code'].str.contains(f'.{element}$'))]

,code,name,name ascii,geonameid
3689,us.ca,california,california,5332921


In [54]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

test_df['geonameid'] = np.nan
test_df['geonameid'] = test_df['elements'].map(lambda elements: infer_geonameid(elements))
#test_df

# Testing one by one
# for e in test_df['elements']:
#     print(e)
#     infer_geonameid(e)

In [55]:
print_geonameid_completeness(test_df)

85.608%


In [56]:
show_nan(test_df)

Number of NaNs: 7545


,tweet_user_location,tweet_id,tweet_user_location_copy,elements,geonameid
67,"Madrid, Comunidad de Madrid",16691,"madrid, comunidad de madrid","[madrid, comunidad de madrid]",NaN
107,Catalunya,10965,catalunya,[catalunya],NaN
110,Text RESIST to 50409,10803,text resist to 50409,[text resist to 50409],NaN
113,Ciudad Autónoma de Buenos Aire,10308,ciudad autónoma de buenos aire,[ciudad autónoma de buenos aire],NaN
173,"Roma, Lazio",7223,"roma, lazio","[roma, lazio]",NaN
...,...,...,...,...,...
19992,Porto Velho RO,45,porto velho ro,[porto velho ro],NaN
19996,Eisha,45,eisha,[eisha],NaN
19997,兵庫県神戸市,45,兵庫県神戸市,[兵庫県神戸市],NaN
19998,Cloud Nine,45,cloud nine,[cloud nine],NaN


In [ ]:
infer_geonameid(['columbus', 'ohio', 'usa'])
infer_geonameid(['usa'])
#get_admin1(admin1_df, 'florida', country_code='US')

In [ ]:
element = 'toronto'
admin1_code = 'on'
# cities_alt_df[(cities_alt_df['altname'] == element) & \
#               (cities_alt_df['admin1'] == admin1_code)]

element = 'england'
country_code = 'GB'
admin1_df[(admin1_df['code'] == f'{country_code}.{element}') & \
         ((admin1_df['name'] == element) | \
          (admin1_df['name ascii'] == element))]



In [ ]:
#countries_df[countries_df['Country'] == 'united states']
infer_geonameid(['united states'])

In [ ]:
elements = ['united states']
get_country(countries_df, elements[0])

## Save file

In [57]:
# Only keep columns we need
output_df = test_df[['tweet_user_location', 'tweet_id', 'geonameid']]
output_path = os.path.join(current_dir, "locations_clean_user_location_geonameid.tsv")
output_df.to_csv(output_path, sep="\t", index=False)

# Special 'with URL' for fast-checking
output_df = output_df.assign(url=lambda x: "https://www.geonames.org/" + x['geonameid'])
output_path = os.path.join(current_dir, "locations_clean_user_location_geonameid_url.tsv")
output_df.to_csv(output_path, sep="\t", index=False)